# RAG (Retrieval-Augmented Generation) 기초 구현

## 📚 개요

RAG는 대규모 언어 모델(LLM)의 한계를 극복하기 위한 기술입니다.

### 🎯 RAG를 사용하는 이유
- **할루시네이션 방지**: 최신 정보나 학습되지 않은 정보에 대한 잘못된 답변 생성 방지
- **정확한 정보 제공**: 실제 문서를 기반으로 한 답변 생성
- **맞춤형 지식 활용**: 특정 도메인이나 조직의 문서를 활용한 답변

### 🔄 RAG 진행 순서
1. **문서 로드** (Document Loading)
2. **텍스트 분할** (Text Splitting) - 청크 단위로 분할
3. **임베딩** (Embedding) - 텍스트를 벡터로 수치화
4. **벡터 저장** (Vector Storage)
5. **검색** (Retrieval) - 질문에 해당하는 문서 검색
6. **답변 생성** (Generation) - 검색된 문서를 기반으로 답변 생성

## 🔧 환경 설정

In [1]:
# 환경 변수 로드
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# 필요한 라이브러리 임포트
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

## 단계 1: 문서 로드 (Load Documents)

PDF 문서를 로드하여 처리 가능한 형태로 변환합니다.

In [3]:
# PyMuPDFLoader를 사용하여 PDF 파일 로드
loader = PyMuPDFLoader("../data/data.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 5


### 📄 문서 내용 확인

In [4]:
# 특정 페이지의 내용 확인 (예: 3번째 페이지)
# print(docs[2].page_content)

### 🏷️ 메타데이터 확인

In [5]:
# 문서의 메타데이터 정보 확인
# docs[2].__dict__

## 단계 2: 문서 분할 (Split Documents)

긴 문서를 작은 청크로 분할하여 검색과 처리를 효율적으로 만듭니다.

In [6]:
# RecursiveCharacterTextSplitter를 사용하여 텍스트 분할
# chunk_size: 각 청크의 최대 크기
# chunk_overlap: 청크 간 중복되는 문자 수 (문맥 유지를 위해)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 25


## 단계 3: 임베딩 (Embedding) 생성

텍스트를 벡터로 변환하여 의미적 유사성을 계산할 수 있게 합니다.

In [7]:
# OpenAI의 임베딩 모델 사용
embeddings = OpenAIEmbeddings()

## 단계 4: 벡터 데이터베이스 생성 및 저장

임베딩된 문서를 벡터 데이터베이스에 저장하여 빠른 검색을 가능하게 합니다.

In [8]:
# FAISS (Facebook AI Similarity Search) 벡터스토어 생성
# 문서와 임베딩을 함께 저장
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

### 🔍 유사도 검색 테스트

In [9]:
# 특정 키워드로 유사한 문서 검색 테스트
# for doc in vectorstore.similarity_search("하향식설계"):
#     print(doc.page_content)
#     print("---")

## 단계 5: 검색기 (Retriever) 생성

벡터스토어를 검색 가능한 형태로 변환합니다.

In [10]:
# 벡터스토어를 검색기로 변환
# 기본적으로 가장 유사한 4개의 문서를 반환
retriever = vectorstore.as_retriever()

### 🧪 검색기 테스트

In [11]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인
# retriever.invoke("요구사항 개발 프로세스의 순서는 무엇인가요?")

## 단계 6: 프롬프트 템플릿 생성

LLM에 전달할 프롬프트의 구조를 정의합니다.

In [12]:
# RAG 프롬프트 템플릿
# context: 검색된 문서 내용
# question: 사용자의 질문
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

## 단계 7: 언어 모델 (LLM) 생성

답변을 생성할 언어 모델을 설정합니다.

In [13]:
# ChatOpenAI 모델 생성
# temperature=0: 일관된 답변을 위해 무작위성 제거
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

## 단계 8: RAG 체인 생성

모든 구성 요소를 연결하여 완전한 RAG 파이프라인을 만듭니다.

In [14]:
# LCEL (LangChain Expression Language)을 사용한 체인 구성
chain = (
    # 입력 처리: context는 retriever에서, question은 그대로 전달
    {"context": retriever, "question": RunnablePassthrough()}
    # 프롬프트 적용
    | prompt
    # LLM으로 답변 생성
    | llm
    # 출력을 문자열로 파싱
    | StrOutputParser()
)

## 🚀 RAG 체인 실행

구성된 RAG 시스템을 사용하여 질문에 답변합니다.

In [15]:
# 질문 설정 및 체인 실행
question = "요구사항의 유형의 비기능적요구는 무엇이 있나요?"
response = chain.invoke(question)

print(f"질문: {question}")
print(f"\n답변: {response}")

질문: 요구사항의 유형의 비기능적요구는 무엇이 있나요?

답변: 비기능적 요구에는 성능, 보안, 품질, 안정성 등이 있습니다.


## 📊 추가 예제

In [16]:
# 다른 질문으로 테스트
questions = [
    "요구사항 개발 프로세스의 순서는 무엇인가요?",
    # "하향식 설계에 대해 설명해주세요.",
    # "소프트웨어 개발 방법론에는 어떤 것들이 있나요?"
]

for q in questions:
    response = chain.invoke(q)
    print(f"\n질문: {q}")
    print(f"답변: {response}")
    print()


질문: 요구사항 개발 프로세스의 순서는 무엇인가요?
답변: 요구사항 개발 프로세스의 순서는 다음과 같습니다:

1. 도출(Elicitation)
2. 분석(Analysis) 
3. 명세(Specification)
4. 확인(Validation)



## 🎯 핵심 개념 정리

### RAG의 주요 구성 요소

1. **Document Loader**: 다양한 형식의 문서를 로드
   - PyMuPDFLoader: PDF 파일 처리
   - 다른 로더들: TextLoader, CSVLoader, JSONLoader 등

2. **Text Splitter**: 긴 문서를 작은 청크로 분할
   - RecursiveCharacterTextSplitter: 가장 일반적인 분할기
   - chunk_size와 chunk_overlap 파라미터로 조정

3. **Embeddings**: 텍스트를 벡터로 변환
   - OpenAIEmbeddings: OpenAI의 임베딩 모델 사용
   - 의미적 유사성 계산 가능

4. **Vector Store**: 임베딩된 문서 저장 및 검색
   - FAISS: 효율적인 유사도 검색
   - 다른 옵션: Chroma, Pinecone, Weaviate 등

5. **Retriever**: 벡터스토어에서 관련 문서 검색
   - 기본적으로 코사인 유사도 기반 검색

6. **LLM Chain**: 검색된 문서를 기반으로 답변 생성
   - 프롬프트 템플릿과 LLM을 연결
   - LCEL로 파이프라인 구성

### 💡 활용 팁

- **청크 크기 조정**: 문서의 특성에 따라 chunk_size 조정
- **오버랩 설정**: chunk_overlap으로 문맥 유지
- **검색 개수 조정**: retriever의 k 파라미터로 검색 문서 수 조정
- **프롬프트 최적화**: 도메인에 맞는 프롬프트 템플릿 작성